# Extracción de Datos de Fondos de Inversión - Santander

Este notebook automatiza la extracción de datos de composición de carteras de fondos de inversión desde los reportes mensuales CAFCI de Santander Argentina.

## Flujo del proceso:
1. **Instalación de dependencias**
2. **Importación de librerías**
3. **Configuración de parámetros**
4. **Navegación web automatizada** (Selenium)
5. **Descarga y extracción de datos del PDF**
6. **Procesamiento y transformación de datos**
7. **Almacenamiento en Data Warehouse**

## 1. Instalación de Dependencias

Instalamos las bibliotecas necesarias para el web scraping, procesamiento de PDFs y manipulación de datos.

In [1]:
%pip install selenium pandas requests pypdf PyMuPDF python-dateutil

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Reiniciar kernel después de instalar paquetes (solo necesario en Databricks)
try:
    dbutils.library.restartPython()
except:
    print("⚠️ No se ejecutó dbutils.library.restartPython() (no estás en Databricks)")

## 1.5 Configuración de Databricks/Spark

Verificar que el entorno Spark está disponible para trabajar con Delta Tables.

In [ ]:
# Verificar que Spark está disponible (solo en Databricks)
try:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()
    print(f"✅ Spark disponible: {spark.version}")
    print(f"✅ Databricks Runtime: {spark.conf.get('spark.databricks.clusterUsageTags.sparkVersion', 'N/A')}")
except Exception as e:
    print(f"⚠️ Advertencia: Spark no disponible ({e})")
    print("   Este notebook está diseñado para ejecutarse en Databricks")
    print("   Si no estás en Databricks, la parte de guardado no funcionará")

## 2. Importación de Librerías

Importamos todas las bibliotecas necesarias para el proceso.

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
import locale
from datetime import datetime

import requests
import io
import re
from pypdf import PdfReader 
import fitz  # PyMuPDF

from typing import Optional, Tuple
import warnings
from contextlib import contextmanager

warnings.filterwarnings('ignore')

# Suprimir warnings específicos de PyMuPDF
import logging
logging.getLogger('fitz').setLevel(logging.ERROR)

@contextmanager
def timeout_context(seconds):
    import threading
    def timeout_handler():
        raise TimeoutError(f"Operación excedió el límite de {seconds} segundos")
    timer = threading.Timer(seconds, timeout_handler)
    timer.start()
    try:
        yield
    finally:
        timer.cancel()

## 3. Configuración de Parámetros

Definimos las constantes y configuraciones del proceso.

In [5]:
class Config:
    URL_FONDO = "https://www.santander.com.ar/empresas/inversiones/informacion-fondos#/detail/12"
    XPATH_REPORTE = "//a[contains(., 'Reporte mensual - CAFCI')]"
    SELENIUM_REMOTE_URL = "https://standalone-chrome-production-c170.up.railway.app/wd/hub"
    USE_REMOTE_CHROME = True
    N_FILAS_ESPERADAS = 10
    TABLA_AREA = [380, 300, 640, 770]
    SOCIEDAD_GERENTE = "Santander AM"
    NOMBRE_FONDO_DEFAULT = "Superfondo Renta Variable - Clase A"
    PATTERN_NOMBRE = re.compile(r'Superfondo\s+(.*?)\s*-\s*Clase\s+\w', re.IGNORECASE | re.DOTALL)
    PATTERN_FECHA = re.compile(r'Datos\s*al\s*(\d{1,2}.*?\d{4})', re.IGNORECASE | re.DOTALL)
    TIMEOUT_IMPLICIT = 10
    TIMEOUT_EXPLICIT = 10
    TABLA_ALMACENAMIENTO = "datos_semanales_bancos"

def configurar_locale():
    locales_spanish = ['es_ES.UTF-8', 'Spanish_Spain', 'es']
    for loc in locales_spanish:
        try:
            locale.setlocale(locale.LC_TIME, loc)
            return True
        except locale.Error:
            continue
    return False

configurar_locale()


True

## 4. Funciones de Utilidad

Definimos funciones reutilizables para cada etapa del proceso.

In [6]:
def obtener_driver() -> webdriver.Chrome:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    try:
        print(f"🌐 Conectando a Chrome remoto (Railway): {Config.SELENIUM_REMOTE_URL}")
        driver = webdriver.Remote(
            command_executor=Config.SELENIUM_REMOTE_URL,
            options=chrome_options
        )
        driver.implicitly_wait(Config.TIMEOUT_IMPLICIT)
        print("✅ Conectado exitosamente a Railway")
        return driver
    except Exception as e:
        print(f"❌ Error conectando a Chrome remoto: {str(e)[:500]}")
        raise RuntimeError("No se pudo conectar al Chrome remoto de Railway.")


def obtener_url_pdf(url: str, xpath: str) -> Optional[str]:
    driver = None
    try:
        driver = obtener_driver()
        driver.set_page_load_timeout(30)
        driver.set_script_timeout(30)
        
        ventana_original = driver.current_window_handle
        
        print(f"🌐 Navegando a {url}")
        driver.get(url)
        time.sleep(2)
        
        wait = WebDriverWait(driver, 15)
        enlace = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        
        driver.execute_script("arguments[0].scrollIntoView(true);", enlace)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", enlace)
        
        wait.until(EC.number_of_windows_to_be(2))
        
        for ventana in driver.window_handles:
            if ventana != ventana_original:
                driver.switch_to.window(ventana)
                break
        
        time.sleep(1)
        pdf_url = driver.current_url
        
        if pdf_url and pdf_url.endswith(".pdf"):
            print(f"✅ PDF encontrado")
            return pdf_url
        else:
            print("❌ URL no termina en .pdf")
            return None
            
    except Exception as e:
        print(f"❌ Error en navegación: {e}")
        return None
        
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass


def descargar_pdf(url: str) -> Optional[io.BytesIO]:
    try:
        print("📥 Descargando PDF...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        pdf_file = io.BytesIO(response.content)
        print(f"✅ Descargado ({len(response.content):,} bytes)")
        return pdf_file
    except Exception as e:
        print(f"❌ Error descargando PDF: {e}")
        return None


def extraer_texto_pdf(pdf_file: io.BytesIO) -> str:
    try:
        pdf_file.seek(0)
        reader = PdfReader(pdf_file)
        return reader.pages[0].extract_text()
    except Exception as e:
        print(f"❌ Error extrayendo texto: {e}")
        return ""


def extraer_nombre_fondo(texto: str) -> str:
    match = Config.PATTERN_NOMBRE.search(texto)
    if match:
        nombre = f"Superfondo {match.group(1).strip()} - Clase A"
        print(f"✅ Fondo: {nombre}")
        return nombre
    else:
        print(f"⚠️ Nombre no encontrado. Usando default")
        return Config.NOMBRE_FONDO_DEFAULT


def extraer_fecha(texto: str) -> Tuple[Optional[datetime], str]:
    from dateutil import parser as dateutil_parser
    
    match = Config.PATTERN_FECHA.search(texto)
    if not match:
        print("❌ Fecha no encontrada en el PDF")
        return None, "Fecha no encontrada"
    
    fecha_str = re.sub(r'\s+', ' ', match.group(1).strip())
    print(f"   Debug: fecha_str extraída = '{fecha_str}'")
    
    # Mapeo de meses en español a número
    meses_es = {
        'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
        'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
        'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
    }
    
    # Intentar parsear manualmente si contiene meses en español
    fecha_lower = fecha_str.lower()
    print(f"   Debug: fecha_lower = '{fecha_lower}'")
    
    for mes_nombre, mes_num in meses_es.items():
        if mes_nombre in fecha_lower:
            print(f"   Debug: Encontrado mes '{mes_nombre}'")
            try:
                # Extraer día y año - usando regex flexible para "de" entre día y mes
                regex_pattern = r'(\d{1,2})\s+(?:de\s+)?' + mes_nombre + r'(?:\s+de)?\s+(\d{4})'
                print(f"   Debug: Intentando regex: {regex_pattern}")
                match_dia_anio = re.search(regex_pattern, fecha_lower)
                if match_dia_anio:
                    dia = match_dia_anio.group(1).zfill(2)
                    anio = match_dia_anio.group(2)
                    # Construir fecha en formato ISO YYYY-MM-DD (más robusto)
                    fecha_formato = f"{anio}-{mes_num}-{dia}"
                    print(f"   Debug: fecha_formato = '{fecha_formato}'")
                    # Usar datetime.strptime que es más robusto que pd.to_datetime en Databricks
                    from datetime import datetime as dt
                    fecha_dt = dt.strptime(fecha_formato, "%Y-%m-%d")
                    print(f"✅ Fecha parseada: {fecha_dt.strftime('%Y-%m-%d')}")
                    return fecha_dt, fecha_str
                else:
                    print(f"   Debug: Regex no coincidió para mes {mes_nombre}")
            except Exception as e:
                print(f"   Debug: Error parsing con mes {mes_nombre}: {e}")
                pass
    
    print(f"   Debug: No se encontró ningún mes en español")
    
    # Si no tiene meses en español, intentar con dateutil (más flexible)
    try:
        # dateutil.parser es muy robusto y maneja muchos formatos
        fecha_dt = dateutil_parser.parse(fecha_str, dayfirst=True)
        print(f"✅ Fecha parseada con dateutil: {fecha_dt.strftime('%Y-%m-%d')}")
        return fecha_dt, fecha_str
    except Exception as e:
        print(f"   Debug: dateutil también falló: {e}")
    
    # Último intento: formatos estándar
    formatos = ["%d de %B %Y", "%d de %B de %Y", "%d/%m/%Y", "%d-%m-%Y"]
    for formato in formatos:
        try:
            fecha_dt = pd.to_datetime(fecha_str, format=formato)
            print(f"✅ Fecha parseada: {fecha_dt.strftime('%Y-%m-%d')}")
            return fecha_dt, fecha_str
        except Exception:
            continue
    
    print(f"❌ No se pudo parsear la fecha: '{fecha_str}'")
    return None, fecha_str


def extraer_valor_cuota_parte(pdf_file: io.BytesIO) -> float:
    try:
        pdf_file.seek(0)
        doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
        page = doc[0]
        
        # Extraer todo el texto
        texto = page.get_text()
        
        # Buscar patrón de valor de cuotaparte
        match = re.search(r'Valor de cuotaparte.*?\$\)\s*([\d.]+[,][\d]+)', texto, re.IGNORECASE | re.DOTALL)
        if match:
            valor_str = match.group(1).strip().replace('.', '').replace(',', '.')
            valor = float(valor_str)
            print(f"✅ Valor cuota parte: ${valor:,.2f}")
            doc.close()
            return valor
        
        doc.close()
        print(f"⚠️ Valor cuota parte no encontrado. Usando default: 1.0")
        return 1.0
    
    except Exception as e:
        print(f"⚠️ Error extrayendo valor cuota parte: {e}. Usando 1.0")
        return 1.0


def extraer_perfil_riesgo(pdf_file: io.BytesIO) -> str:
    try:
        pdf_file.seek(0)
        doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
        page = doc[0]
        
        # Extraer todo el texto
        texto = page.get_text()
        
        # Buscar patrón de perfil de riesgo
        match = re.search(r'Perfil de riesgo\s+(\w+)', texto, re.IGNORECASE)
        if match:
            perfil = match.group(1).strip()
            print(f"✅ Perfil de riesgo: {perfil}")
            doc.close()
            return perfil
        
        doc.close()
        print(f"⚠️ Perfil de riesgo no encontrado. Usando default: Alto")
        return 'Alto'
    
    except Exception as e:
        print(f"⚠️ Error extrayendo perfil riesgo: {e}. Usando Alto")
        return 'Alto'


def extraer_tabla_composicion_v2(pdf_file: io.BytesIO, n_filas: int = 10) -> Optional[pd.DataFrame]:
    """
    Extrae la composición de la cartera (acciones individuales) del PDF usando PyMuPDF.
    
    Estrategia:
    1. Busca porcentajes en el área derecha (x > 480)
    2. Para cada porcentaje, busca la línea completa de texto en la misma Y
    3. Extrae acción completa (nombre + ticker) de esa línea
    """
    try:
        pdf_file.seek(0)
        doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
        page = doc[0]
        
        # Extraer palabras con posiciones
        words = page.get_text("words")  # [(x0, y0, x1, y1, "word", block_no, line_no, word_no)]
        
        if not words:
            print("❌ No se pudieron extraer palabras del PDF")
            doc.close()
            return None
        
        print("📄 Extrayendo composición de acciones del PDF con PyMuPDF...")
        
        # PASO 1: Buscar porcentajes en área derecha (x > 480, y: 420-580)
        porcentajes_encontrados = []
        for w in words:
            x0, y0, x1, y1, text, *_ = w
            if x0 > 480 and 420 < y0 < 580:
                text_clean = text.strip().replace('%', '')
                if text_clean.isdigit() and 1 <= int(text_clean) <= 99:
                    porcentajes_encontrados.append({
                        'pct': int(text_clean),
                        'y': y0,
                        'x': x0
                    })
        
        porcentajes_encontrados.sort(key=lambda x: x['y'])
        
        print(f"   Porcentajes encontrados: {len(porcentajes_encontrados)}")
        
        if not porcentajes_encontrados:
            print("❌ No se encontraron porcentajes en el área esperada")
            doc.close()
            return None
        
        # PASO 2: Extraer líneas completas de texto con get_text("dict")
        text_dict = page.get_text("dict")
        lineas_completas = []
        
        for block in text_dict['blocks']:
            if 'lines' in block:
                for line in block['lines']:
                    # Unir todos los spans de una línea
                    line_text = ""
                    line_y = None
                    line_x0 = 999999
                    
                    for span in line['spans']:
                        bbox = span['bbox']
                        y = bbox[1]
                        x = bbox[0]
                        
                        # Área de composición: nombres de acciones están entre x: 300-450
                        if 420 < y < 580 and 300 < x < 460:
                            line_text += span['text']
                            if line_y is None:
                                line_y = y
                            line_x0 = min(line_x0, x)
                    
                    if line_text.strip() and line_y:
                        # Filtrar líneas que sean solo números o porcentajes
                        texto = line_text.strip()
                        # Ignorar si es solo números (4000, 2000, etc.)
                        if texto.isdigit():
                            continue
                        # Ignorar si es solo porcentaje (40%, 60%, etc.)
                        if texto.replace('%', '').strip().isdigit():
                            continue
                        # Ignorar líneas muy cortas (menos de 3 caracteres)
                        if len(texto) < 3:
                            continue
                        
                        lineas_completas.append({
                            'texto': texto,
                            'y': line_y,
                            'x': line_x0
                        })
        
        lineas_completas.sort(key=lambda x: x['y'])
        
        # PASO 3: MAPEO - Unir porcentajes con líneas completas por posición Y
        composicion = []
        tolerancia_y = 5  # Tolerancia de 5 puntos en coordenada Y
        
        for pct_data in porcentajes_encontrados:
            pct_y = pct_data['y']
            porcentaje = pct_data['pct']
            
            # Buscar línea de texto cercana
            accion_completa = None
            for linea in lineas_completas:
                if abs(linea['y'] - pct_y) < tolerancia_y:
                    accion_completa = linea['texto']
                    break
            
            # Si no encontramos línea, buscar en palabras individuales
            if not accion_completa:
                palabras_linea = []
                for w in words:
                    x0, y0, x1, y1, text, *_ = w
                    # Área de nombres de acciones: x entre 300-460
                    if abs(y0 - pct_y) < tolerancia_y and 300 < x0 < 460:
                        # Filtrar números puros y porcentajes
                        if not text.strip().isdigit() and not text.strip().replace('%', '').isdigit():
                            palabras_linea.append((x0, text))
                
                if palabras_linea:
                    palabras_linea.sort(key=lambda x: x[0])
                    accion_completa = ' '.join([p[1] for p in palabras_linea])
            
            if accion_completa:
                composicion.append({
                    'Accion': accion_completa,
                    'Porcentaje': f"{porcentaje}%"
                })
            else:
                print(f"   ⚠️ No se encontró acción para porcentaje {porcentaje}% en y={pct_y:.1f}")
        
        doc.close()
        
        if not composicion:
            print("❌ No se pudieron mapear acciones con porcentajes")
            return None
        
        df_resultado = pd.DataFrame(composicion)
        
        # Validar total
        total_porcentaje = sum(int(d['Porcentaje'].rstrip('%')) for d in composicion)
        
        print(f"✅ Composición extraída: {len(df_resultado)} acciones")
        print(f"   Total cartera: {total_porcentaje}%")
        
        if total_porcentaje < 80 or total_porcentaje > 110:
            print(f"   ⚠️ Advertencia: Total parece incorrecto ({total_porcentaje}%)")
        
        # Mostrar Top 3
        print(f"\n   Top 3 acciones:")
        for idx, row in df_resultado.head(3).iterrows():
            print(f"      {row['Accion']}: {row['Porcentaje']}")
        
        return df_resultado
    
    except Exception as e:
        print(f"❌ Error extrayendo composición: {e}")
        import traceback
        traceback.print_exc()
        return None


def procesar_dataframe(df_composicion: pd.DataFrame, fecha: Optional[datetime], nombre_fondo: str, sociedad_gerente: str, valor_cuota_parte: float = 1.0, perfil_riesgo: str = 'Agresivo') -> pd.DataFrame:
    try:
        if df_composicion is None or df_composicion.empty:
            print("⚠️ DataFrame de composición vacío")
            return pd.DataFrame()
        
        df = df_composicion.copy()
        df['Porcentaje'] = df['Porcentaje'].str.rstrip('%').astype(float) / 100.0
        
        df['Periodo_x'] = fecha if fecha else None
        df['Nombre_Fondo'] = nombre_fondo
        df['Sociedad_Gerente'] = sociedad_gerente
        df['Perfil_de_Inversor'] = perfil_riesgo
        df['Valor_Cuota_Parte'] = valor_cuota_parte
        
        columnas_finales = ['Periodo_x', 'Nombre_Fondo', 'Sociedad_Gerente', 'Accion', 'Porcentaje', 'Perfil_de_Inversor', 'Valor_Cuota_Parte']
        df = df[columnas_finales]
        
        print(f"✅ DataFrame procesado: {len(df)} registros")
        print(f"   Total cartera: {df['Porcentaje'].sum():.1%}")
        print(f"   Perfil: {perfil_riesgo}")
        print(f"   Valor cuota parte: ${valor_cuota_parte:,.2f}")
        
        return df
    
    except Exception as e:
        print(f"❌ Error procesando DataFrame: {e}")
        return pd.DataFrame()


def guardar_en_databricks(df: pd.DataFrame, tabla: str, merge: bool = True) -> bool:
    try:
        try:
            from pyspark.sql import SparkSession
            spark = SparkSession.builder.getOrCreate()
        except ImportError:
            print("⚠️ PySpark no disponible. Este código debe ejecutarse en Databricks")
            return False
        
        if df is None or df.empty:
            print("⚠️ DataFrame vacío, no hay nada que guardar")
            return False
        
        columnas_esperadas = ['Periodo_x', 'Nombre_Fondo', 'Sociedad_Gerente', 'Accion', 'Porcentaje', 'Perfil_de_Inversor', 'Valor_Cuota_Parte']
        columnas_faltantes = set(columnas_esperadas) - set(df.columns)
        if columnas_faltantes:
            print(f"❌ Faltan columnas requeridas: {columnas_faltantes}")
            return False
        
        spark_df = spark.createDataFrame(df)
        
        if merge:
            from delta.tables import DeltaTable
            
            if DeltaTable.isDeltaTable(spark, tabla):
                print(f"📝 Haciendo MERGE en '{tabla}'...")
                delta_table = DeltaTable.forName(spark, tabla)
                delta_table.alias("target").merge(
                    spark_df.alias("source"),
                    "target.Periodo_x = source.Periodo_x AND target.Accion = source.Accion"
                ).whenMatchedUpdateAll() \
                 .whenNotMatchedInsertAll() \
                 .execute()
                print("✅ MERGE completado")
            else:
                print(f"🆕 Creando tabla '{tabla}'...")
                spark_df.write.format("delta").mode("overwrite").saveAsTable(tabla)
                print("✅ Tabla creada")
        else:
            print(f"📝 Haciendo APPEND en '{tabla}'...")
            spark_df.write.format("delta").mode("append").saveAsTable(tabla)
            print("✅ APPEND completado")
        
        count = spark.table(tabla).count()
        print(f"📊 Total registros en tabla: {count:,}")
        return True
        
    except Exception as e:
        print(f"❌ Error guardando en Databricks: {e}")
        return False


def mostrar_resumen(df: pd.DataFrame, nombre_fondo: str, fecha: Optional[datetime], pdf_url: str):
    print("\n" + "=" * 80)
    print("📋 RESUMEN EJECUTIVO")
    print("=" * 80)
    
    print(f"\n🏢 Fondo:           {nombre_fondo}")
    print(f"🏦 Gerente:         {Config.SOCIEDAD_GERENTE}")
    print(f"📅 Periodo:         {fecha.strftime('%Y-%m-%d') if fecha else 'Sin fecha'}")
    print(f"📄 PDF:             {pdf_url[:70]}..." if pdf_url else "No disponible")
    
    if df is not None and not df.empty:
        print(f"\n✅ Estado:          Extracción exitosa")
        print(f"📊 Registros:       {len(df)}")
        print(f"💹 Total cartera:   {df['Porcentaje'].sum():.2%}")
        
        print("\n🔝 Top 5 holdings:")
        for idx, row in df.nlargest(5, 'Porcentaje').iterrows():
            accion = row['Accion'][:45]
            pct = row['Porcentaje']
            print(f"   {accion:45s} {pct:>7.2%}")
    else:
        print(f"\n⚠️ Estado:          Sin datos extraídos")
    
    print("\n" + "=" * 80)

## 5. Ejecución Principal

Proceso completo de extracción, transformación y carga (ETL).


In [7]:
def ejecutar_pipeline_completo():
    print("🚀 Iniciando pipeline de extracción...\n")
    
    pdf_url = obtener_url_pdf(Config.URL_FONDO, Config.XPATH_REPORTE)
    if not pdf_url:
        print("❌ Pipeline abortado: No se pudo obtener la URL del PDF")
        return None
    
    pdf_file = descargar_pdf(pdf_url)
    if not pdf_file:
        print("❌ Pipeline abortado: No se pudo descargar el PDF")
        return None
    
    texto = extraer_texto_pdf(pdf_file)
    if not texto:
        print("❌ Pipeline abortado: No se pudo extraer texto del PDF")
        return None
    
    nombre_fondo = extraer_nombre_fondo(texto)
    fecha, fecha_str = extraer_fecha(texto)
    
    if fecha is None:
        print(f"⚠️ Advertencia: No se pudo parsear la fecha '{fecha_str}'")
        print("   El pipeline continuará pero la columna Periodo_x será NULL")
    
    valor_cuota_parte = extraer_valor_cuota_parte(pdf_file)
    perfil_riesgo = extraer_perfil_riesgo(pdf_file)
    
    df_composicion = extraer_tabla_composicion_v2(pdf_file, Config.N_FILAS_ESPERADAS)
    if df_composicion is None or df_composicion.empty:
        print("❌ Pipeline abortado: No se pudo extraer la tabla de composición")
        return None
    
    print(f"✅ Composición extraída: {len(df_composicion)} registros")
    
    df_final = procesar_dataframe(
        df_composicion,
        fecha,
        nombre_fondo,
        Config.SOCIEDAD_GERENTE,
        valor_cuota_parte,
        perfil_riesgo
    )
    
    if df_final.empty:
        print("❌ Pipeline abortado: DataFrame final está vacío después del procesamiento")
        return None
    
    mostrar_resumen(df_final, nombre_fondo, fecha, pdf_url)
    
    print("\n✅ Pipeline completado exitosamente")
    return df_final


df_resultado = ejecutar_pipeline_completo()


🚀 Iniciando pipeline de extracción...

🌐 Conectando a Chrome remoto (Railway): https://standalone-chrome-production-c170.up.railway.app/wd/hub
✅ Conectado exitosamente a Railway
✅ Conectado exitosamente a Railway
🌐 Navegando a https://www.santander.com.ar/empresas/inversiones/informacion-fondos#/detail/12
🌐 Navegando a https://www.santander.com.ar/empresas/inversiones/informacion-fondos#/detail/12
✅ PDF encontrado
✅ PDF encontrado
📥 Descargando PDF...
📥 Descargando PDF...
✅ Descargado (393,456 bytes)
✅ Descargado (393,456 bytes)
✅ Fondo: Superfondo Renta Variable - Clase A
   Debug: fecha_str extraída = '31 de octubre 2025'
   Debug: fecha_lower = '31 de octubre 2025'
   Debug: Encontrado mes 'octubre'
   Debug: Intentando regex: (\d{1,2})\s+(?:de\s+)?octubre(?:\s+de)?\s+(\d{4})
   Debug: fecha_formato = '2025-10-31'
✅ Fecha parseada: 2025-10-31
✅ Valor cuota parte: $5,094.84
✅ Perfil de riesgo: Plazo
📄 Extrayendo composición de acciones del PDF con PyMuPDF...
   Porcentajes encontrado

In [8]:
# Mostrar el DataFrame completo si existe
if df_resultado is not None and not df_resultado.empty:
    print("📊 Datos extraídos:\n")
    display(df_resultado)
    
    print(f"\n📈 Estadísticas:")
    print(f"   - Total de holdings: {len(df_resultado)}")
    print(f"   - Suma de porcentajes: {df_resultado['Porcentaje'].sum():.2%}")
    print(f"   - Mayor holding: {df_resultado.loc[df_resultado['Porcentaje'].idxmax(), 'Accion']}")
    print(f"   - % del mayor: {df_resultado['Porcentaje'].max():.2%}")
else:
    print("⚠️ No hay datos para mostrar")


📊 Datos extraídos:



,Periodo_x,Nombre_Fondo,Sociedad_Gerente,Accion,Porcentaje,Perfil_de_Inversor,Valor_Cuota_Parte
0,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Grup.Financ.Galicia - (GGAL-AR) -,0.17,Plazo,5094.83724
1,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Pampa Holding S.A. - (PAMP-AR),0.12,Plazo,5094.83724
2,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,YPF - (YPFD-AR) -,0.12,Plazo,5094.83724
3,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Banco Macro SA B (BMA-AR),0.09,Plazo,5094.83724
4,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Bolsas y Mercados Argentinos S.A. (BYMA…,0.07,Plazo,5094.83724
5,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Banco Frances - (BBAR-AR) -,0.07,Plazo,5094.83724
6,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Central Puerto S.A. - (CEPU-AR),0.06,Plazo,5094.83724
7,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Grupo Supervielle SUPV AR,0.05,Plazo,5094.83724
8,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Transportadora Gas del Sur - (TGSU2-AR) -,0.05,Plazo,5094.83724
9,2025-10-31,Superfondo Renta Variable - Clase A,Santander AM,Siderar - (TXAR-AR) -,0.04,Plazo,5094.83724



📈 Estadísticas:
   - Total de holdings: 10
   - Suma de porcentajes: 84.00%
   - Mayor holding: Grup.Financ.Galicia - (GGAL-AR) -
   - % del mayor: 17.00%


In [9]:
if df_resultado is not None and not df_resultado.empty:
    exito = guardar_en_databricks(
        df=df_resultado,
        tabla=Config.TABLA_ALMACENAMIENTO,
        merge=True
    )
    
    if exito:
        print("\n🎉 Datos guardados exitosamente en Data Warehouse")
        print(f"📊 Tabla: {Config.TABLA_ALMACENAMIENTO}")
        print(f"📈 Registros guardados: {len(df_resultado)}")
    else:
        print("\n⚠️ Hubo un problema al guardar los datos")
else:
    print("⚠️ No hay datos para guardar")


⚠️ PySpark no disponible. Este código debe ejecutarse en Databricks

⚠️ Hubo un problema al guardar los datos
